In [36]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter
from langsmith import wrappers, Client

In [37]:
import os

#os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "tracing-demo-2"

In [38]:
from getpass import getpass

os.environ["LANGSMITH_API_KEY"] = getpass("Enter your LangSmith API Key: ")


In [39]:
loader = TextLoader("../00-example_data/state_of_the_union.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
texts

In [41]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    #model="llama3.2:latest",
    model="nomic-embed-text:latest",
    base_url="http://localhost:11434"  # Ensure Ollama is running locally
)

#embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

In [42]:
retriever = vectorstore.as_retriever()

In [43]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

In [ ]:
docs

### Maximum marginal relevance retrieval

In [44]:
retriever = vectorstore.as_retriever(search_type="mmr")

In [45]:
mmr_docs = retriever.invoke("what did the president say about ketanji brown jackson?")

In [ ]:
mmr_docs

### Passing search parameters
#### Similarity score threshold retrieval

In [27]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.3}
)

In [28]:
scr_threshold_docs = retriever.invoke("what did the president say about ketanji brown jackson?")

In [ ]:
scr_threshold_docs

#### Specifying top k

In [30]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [31]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")

In [ ]:
docs